# Experimentos

In [1]:
# imports
import numpy as np
import pandas as pd

# time
import time
from datetime import timedelta

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ETL
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Oversampling Methods
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# Deep Learning Framework - Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

# K-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

models = pd.DataFrame()

Using TensorFlow backend.


#  Raw Data

In [2]:
## Data
df = pd.read_csv('data/train.csv')

## Remove duplicate rows
df.drop_duplicates(inplace=True)

# Slit (X,Y)
x_data = df.drop(['ID','TARGET'], axis=1)
y_data = df['TARGET'].copy()

x_data.shape

(76020, 369)

# Oversampling: Random Oversampling

## 2 CNN 1 MAX RO

In [3]:
# info
model_name = '2 CNN 1 MAX RO'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97550 samples, validate on 13684 samples
Epoch 1/10000
97550/97550 [==============================] - 44s - loss: 0.5680 - acc: 0.7103 - val_loss: 0.4962 - val_acc: 0.7539
Epoch 2/10000
97550/97550 [==============================] - 19s - loss: 0.4988 - acc: 0.7548 - val_loss: 0.5279 - val_acc: 0.7154
Epoch 3/10000
97550/97550 [==============================] - 19s - loss: 0.4748 - acc: 0.7696 - val_loss: 0.5152 - val_acc: 0.7360
Epoch 4/10000
97550/97550 [==============================] - 18s - loss: 0.4566 - acc: 0.7803 - val_loss: 0.5561 - val_acc: 0.7110
Epoch 5/10000
97550/97550 [==============================] - 18s - loss: 0.4397 - acc: 0.7907 - val_loss: 0.4694 - val_acc: 0.7735
Epoch 6/10000
97550/97550 [==============================] - 19s - loss: 0.4281 - acc: 0.7989 - val_loss: 0.6193 - val_acc: 0.6855
Epoch 7/10000
97550/97550 [==============================] - 18s - loss: 0.4145 - acc: 0.8070 - val_loss: 0.5161 - val_acc: 0.7397
Epoch 8/10000
97550/97550 [=

97526/97526 [==============================] - 18s - loss: 0.3129 - acc: 0.8616 - val_loss: 0.4446 - val_acc: 0.8296
Epoch 22/10000
97526/97526 [==============================] - 18s - loss: 0.3041 - acc: 0.8635 - val_loss: 0.4773 - val_acc: 0.8091
Epoch 23/10000
97526/97526 [==============================] - 18s - loss: 0.3004 - acc: 0.8649 - val_loss: 0.4829 - val_acc: 0.8078
Epoch 24/10000
97526/97526 [==============================] - 18s - loss: 0.2981 - acc: 0.8674 - val_loss: 0.5092 - val_acc: 0.7700
Epoch 25/10000
97526/97526 [==============================] - 18s - loss: 0.2980 - acc: 0.8680 - val_loss: 0.4603 - val_acc: 0.8267
Epoch 26/10000
97526/97526 [==============================] - 19s - loss: 0.3555 - acc: 0.8497 - val_loss: 0.4665 - val_acc: 0.8156
Epoch 27/10000
97526/97526 [==============================] - 18s - loss: 0.3228 - acc: 0.8604 - val_loss: 0.4471 - val_acc: 0.8356
Epoch 28/10000
97526/97526 [==============================] - 18s - loss: 0.3061 - acc: 0.8

97670/97670 [==============================] - 18s - loss: 0.3064 - acc: 0.8669 - val_loss: 0.4658 - val_acc: 0.8321
Epoch 24/10000
97670/97670 [==============================] - 18s - loss: 0.3067 - acc: 0.8685 - val_loss: 0.4509 - val_acc: 0.8343
Epoch 25/10000
97670/97670 [==============================] - 18s - loss: 0.3054 - acc: 0.8687 - val_loss: 0.4572 - val_acc: 0.8297
Epoch 26/10000
97670/97670 [==============================] - 18s - loss: 0.3003 - acc: 0.8707 - val_loss: 0.4631 - val_acc: 0.8275
Epoch 27/10000
97472/97670 [============================>.] - ETA: 0sKF_9
Train on 97438 samples, validate on 13684 samples
Epoch 1/10000
97438/97438 [==============================] - 19s - loss: 0.5523 - acc: 0.7243 - val_loss: 0.4734 - val_acc: 0.7713
Epoch 2/10000
97438/97438 [==============================] - 18s - loss: 0.4936 - acc: 0.7643 - val_loss: 0.5753 - val_acc: 0.7031
Epoch 3/10000
97438/97438 [==============================] - 18s - loss: 0.4711 - acc: 0.7767 - val_l

In [4]:
print(model_name)

metrics_kf

2 CNN 1 MAX RO


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.957286,0.718334,0.718334,0.553008,0.690167,0.558537,0.839274,00:14:05.595090
KF_2,0.945507,0.744849,0.744849,0.558550,0.724351,0.563873,0.831514,00:09:07.743407
KF_3,0.950722,0.738092,0.738092,0.555717,0.707077,0.561067,0.835043,00:08:50.909838
KF_4,0.949989,0.751846,0.751846,0.551558,0.718384,0.547266,0.807814,00:11:34.691106
KF_5,0.948446,0.720919,0.720919,0.546064,0.687472,0.541588,0.812681,00:08:19.033775
KF_6,0.948707,0.746041,0.746041,0.553063,0.721894,0.550435,0.811497,00:08:56.436855
KF_7,0.951152,0.758056,0.761378,0.551967,0.720113,0.547886,0.808077,00:07:41.592113
KF_8,0.951720,0.684338,0.684338,0.549399,0.673391,0.554002,0.840700,00:08:29.267768
KF_9,0.953770,0.735690,0.735690,0.558075,0.708594,0.566195,0.842389,00:10:19.528150
KF_10,0.947752,0.729735,0.729735,0.547818,0.710675,0.538836,0.797264,00:08:00.766999


## 2 CNN 2 MAX RO

In [5]:
# info
model_name = '2 CNN 2 MAX RO'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(94, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97536 samples, validate on 13684 samples
Epoch 1/10000
97536/97536 [==============================] - 21s - loss: 0.6421 - acc: 0.6558 - val_loss: 0.6455 - val_acc: 0.6964
Epoch 2/10000
97536/97536 [==============================] - 20s - loss: 0.6139 - acc: 0.6845 - val_loss: 0.5805 - val_acc: 0.6923
Epoch 3/10000
97536/97536 [==============================] - 20s - loss: 0.6047 - acc: 0.6910 - val_loss: 0.5790 - val_acc: 0.7038
Epoch 4/10000
97536/97536 [==============================] - 20s - loss: 0.5969 - acc: 0.6963 - val_loss: 0.6096 - val_acc: 0.6600
Epoch 5/10000
97536/97536 [==============================] - 20s - loss: 0.5890 - acc: 0.7000 - val_loss: 0.6390 - val_acc: 0.6603
Epoch 6/10000
97536/97536 [==============================] - 20s - loss: 0.5799 - acc: 0.7071 - val_loss: 0.5535 - val_acc: 0.6932
Epoch 7/10000
97536/97536 [==============================] - 20s - loss: 0.5400 - acc: 0.7340 - val_loss: 0.5260 - val_acc: 0.7309
Epoch 8/10000
97536/97536 [=

97518/97518 [==============================] - 20s - loss: 0.4613 - acc: 0.7772 - val_loss: 0.4750 - val_acc: 0.7375
Epoch 25/10000
97518/97518 [==============================] - 20s - loss: 0.4568 - acc: 0.7810 - val_loss: 0.4767 - val_acc: 0.7497
Epoch 26/10000
97518/97518 [==============================] - 20s - loss: 0.4557 - acc: 0.7815 - val_loss: 0.5117 - val_acc: 0.7468
Epoch 27/10000
97518/97518 [==============================] - 20s - loss: 0.4546 - acc: 0.7818 - val_loss: 0.5136 - val_acc: 0.7388
Epoch 28/10000
97518/97518 [==============================] - 20s - loss: 0.4527 - acc: 0.7826 - val_loss: 0.5195 - val_acc: 0.7586
Epoch 29/10000
97518/97518 [==============================] - 20s - loss: 0.4550 - acc: 0.7825 - val_loss: 0.4998 - val_acc: 0.7306
Epoch 30/10000
97518/97518 [==============================] - 20s - loss: 0.4577 - acc: 0.7809 - val_loss: 0.5341 - val_acc: 0.7453
Epoch 31/10000
97518/97518 [==============================] - 20s - loss: 0.4517 - acc: 0.7

97430/97430 [==============================] - 20s - loss: 0.4585 - acc: 0.7804 - val_loss: 0.5128 - val_acc: 0.7599
Epoch 39/10000
97430/97430 [==============================] - 21s - loss: 0.4648 - acc: 0.7788 - val_loss: 0.5445 - val_acc: 0.7612
Epoch 40/10000
97430/97430 [==============================] - 19s - loss: 0.4605 - acc: 0.7814 - val_loss: 0.4826 - val_acc: 0.7904
Epoch 41/10000
97430/97430 [==============================] - 20s - loss: 0.4530 - acc: 0.7854 - val_loss: 0.5015 - val_acc: 0.7781
Epoch 42/10000
97430/97430 [==============================] - 20s - loss: 0.4531 - acc: 0.7853 - val_loss: 0.4792 - val_acc: 0.7796
Epoch 43/10000
97430/97430 [==============================] - 20s - loss: 0.4570 - acc: 0.7835 - val_loss: 0.4579 - val_acc: 0.7751
Epoch 44/10000
97280/97430 [============================>.] - ETA: 0sKF_6
Train on 97460 samples, validate on 13684 samples
Epoch 1/10000
97460/97460 [==============================] - 20s - loss: 0.6380 - acc: 0.6598 - val

97610/97610 [==============================] - 20s - loss: 0.4441 - acc: 0.7884 - val_loss: 0.4959 - val_acc: 0.7567
Epoch 36/10000
97610/97610 [==============================] - 20s - loss: 0.4426 - acc: 0.7888 - val_loss: 0.5145 - val_acc: 0.7423
Epoch 37/10000
97610/97610 [==============================] - 20s - loss: 0.4424 - acc: 0.7893 - val_loss: 0.4980 - val_acc: 0.7625
Epoch 38/10000
97610/97610 [==============================] - 20s - loss: 0.4403 - acc: 0.7896 - val_loss: 0.4944 - val_acc: 0.7413
Epoch 39/10000
97610/97610 [==============================] - 20s - loss: 0.4414 - acc: 0.7896 - val_loss: 0.4809 - val_acc: 0.7470
Epoch 40/10000
97610/97610 [==============================] - 20s - loss: 0.4435 - acc: 0.7899 - val_loss: 0.5160 - val_acc: 0.7523
Epoch 41/10000
97610/97610 [==============================] - 20s - loss: 0.4387 - acc: 0.7920 - val_loss: 0.4732 - val_acc: 0.7615
Epoch 42/10000
97610/97610 [==============================] - 20s - loss: 0.4433 - acc: 0.7

In [6]:
print(model_name)

metrics_kf

2 CNN 2 MAX RO


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.907018,0.755414,0.758737,0.537220,0.710557,0.490709,0.704064,00:33:20.544805
KF_2,0.891030,0.804581,0.804581,0.548442,0.747931,0.524008,0.754439,00:12:16.902418
KF_3,0.880523,0.782137,0.782137,0.538002,0.712368,0.494267,0.710602,00:13:35.606968
KF_4,0.871885,0.797628,0.800951,0.550134,0.737242,0.535697,0.779795,00:10:37.565274
KF_5,0.883678,0.764173,0.764173,0.542232,0.718186,0.513354,0.746251,00:14:55.023949
KF_6,0.883237,0.789212,0.789212,0.548287,0.739128,0.527912,0.765062,00:14:26.227526
KF_7,0.883142,0.797833,0.801155,0.541399,0.728242,0.501844,0.719679,00:13:14.872914
KF_8,0.882214,0.785910,0.785910,0.544834,0.729163,0.518724,0.752039,00:15:02.014122
KF_9,0.870544,0.799998,0.799998,0.544588,0.740840,0.510614,0.732404,00:08:46.661627
KF_10,0.898789,0.774018,0.774018,0.542802,0.727461,0.510508,0.737403,00:24:38.873090


# Oversampling: SMOTE

## 2 CNN 1 MAX SM

In [7]:
# info
model_name = '2 CNN 1 MAX SM'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97552 samples, validate on 13684 samples
Epoch 1/10000
97552/97552 [==============================] - 19s - loss: 0.4149 - acc: 0.8215 - val_loss: 0.2777 - val_acc: 0.8793
Epoch 2/10000
97552/97552 [==============================] - 18s - loss: 0.3250 - acc: 0.8672 - val_loss: 0.3390 - val_acc: 0.8720
Epoch 3/10000
97552/97552 [==============================] - 18s - loss: 0.3005 - acc: 0.8799 - val_loss: 0.2397 - val_acc: 0.8864
Epoch 4/10000
97552/97552 [==============================] - 18s - loss: 0.2862 - acc: 0.8882 - val_loss: 0.3012 - val_acc: 0.8486
Epoch 5/10000
97552/97552 [==============================] - 18s - loss: 0.2777 - acc: 0.8911 - val_loss: 0.4054 - val_acc: 0.8267
Epoch 6/10000
97552/97552 [==============================] - 18s - loss: 0.2668 - acc: 0.8968 - val_loss: 0.3606 - val_acc: 0.8476
Epoch 7/10000
97552/97552 [==============================] - 18s - loss: 0.2598 - acc: 0.8998 - val_loss: 0.2395 - val_acc: 0.9016
Epoch 8/10000
97552/97552 [=

97517/97517 [==============================] - 18s - loss: 0.2872 - acc: 0.8813 - val_loss: 0.2938 - val_acc: 0.8742
Epoch 5/10000
97517/97517 [==============================] - 19s - loss: 0.2777 - acc: 0.8870 - val_loss: 0.3954 - val_acc: 0.8397
Epoch 6/10000
97517/97517 [==============================] - 19s - loss: 0.2698 - acc: 0.8910 - val_loss: 0.2777 - val_acc: 0.8774
Epoch 7/10000
97517/97517 [==============================] - 19s - loss: 0.2593 - acc: 0.8962 - val_loss: 0.3784 - val_acc: 0.8444
Epoch 8/10000
97517/97517 [==============================] - 19s - loss: 0.2539 - acc: 0.8981 - val_loss: 0.3122 - val_acc: 0.8643
Epoch 9/10000
97517/97517 [==============================] - 18s - loss: 0.2478 - acc: 0.9011 - val_loss: 0.2965 - val_acc: 0.8714
Epoch 10/10000
97517/97517 [==============================] - 19s - loss: 0.2440 - acc: 0.9031 - val_loss: 0.3307 - val_acc: 0.8542
Epoch 11/10000
97517/97517 [==============================] - 19s - loss: 0.2386 - acc: 0.9054 -

97590/97590 [==============================] - 18s - loss: 0.2062 - acc: 0.9185 - val_loss: 0.3134 - val_acc: 0.8708
Epoch 25/10000
97590/97590 [==============================] - 18s - loss: 0.2048 - acc: 0.9195 - val_loss: 0.3132 - val_acc: 0.8734
Epoch 26/10000
97590/97590 [==============================] - 18s - loss: 0.2028 - acc: 0.9195 - val_loss: 0.3167 - val_acc: 0.8726
Epoch 27/10000
97590/97590 [==============================] - 19s - loss: 0.2024 - acc: 0.9205 - val_loss: 0.3148 - val_acc: 0.8820
Epoch 28/10000
97590/97590 [==============================] - 19s - loss: 0.1989 - acc: 0.9214 - val_loss: 0.3830 - val_acc: 0.8313
Epoch 29/10000
97590/97590 [==============================] - 18s - loss: 0.1989 - acc: 0.9204 - val_loss: 0.3707 - val_acc: 0.8555
Epoch 30/10000
97312/97590 [============================>.] - ETA: 0sKF_9
Train on 97505 samples, validate on 13684 samples
Epoch 1/10000
97505/97505 [==============================] - 19s - loss: 0.4448 - acc: 0.8053 - val

In [8]:
print(model_name)

metrics_kf

2 CNN 1 MAX SM


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.977915,0.761836,0.761836,0.566660,0.681202,0.585047,0.880179,00:12:09.157658
KF_2,0.972980,0.795865,0.795865,0.573897,0.729669,0.593809,0.869262,00:07:10.167446
KF_3,0.974347,0.767786,0.771108,0.562225,0.701748,0.575650,0.858458,00:07:52.424997
KF_4,0.976265,0.776836,0.776836,0.564890,0.698237,0.580924,0.867009,00:09:21.341207
KF_5,0.978595,0.755739,0.759061,0.560910,0.687311,0.574961,0.864378,00:13:02.648688
KF_6,0.976559,0.775616,0.775616,0.556684,0.701494,0.564406,0.842673,00:10:18.201702
KF_7,0.979011,0.776975,0.776975,0.567893,0.708272,0.585245,0.867929,00:09:28.612699
KF_8,0.976283,0.743163,0.746486,0.563020,0.685615,0.578840,0.870297,00:09:36.327837
KF_9,0.975056,0.783297,0.783297,0.569305,0.699026,0.588378,0.876200,00:07:44.890210
KF_10,0.978844,0.735398,0.735398,0.568617,0.698478,0.587292,0.875148,00:11:50.266775


## 2 CNN 2 MAX SM

In [9]:
# info
model_name = '2 CNN 2 MAX SM'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(94, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97538 samples, validate on 13684 samples
Epoch 1/10000
97538/97538 [==============================] - 21s - loss: 0.5486 - acc: 0.7593 - val_loss: 0.4348 - val_acc: 0.8755
Epoch 2/10000
97538/97538 [==============================] - 20s - loss: 0.4509 - acc: 0.7944 - val_loss: 0.3576 - val_acc: 0.7425
Epoch 3/10000
97538/97538 [==============================] - 20s - loss: 0.4100 - acc: 0.8055 - val_loss: 0.3525 - val_acc: 0.7560
Epoch 4/10000
97538/97538 [==============================] - 20s - loss: 0.3819 - acc: 0.8228 - val_loss: 1.7467 - val_acc: 0.5779
Epoch 5/10000
97538/97538 [==============================] - 20s - loss: 0.3870 - acc: 0.8162 - val_loss: 2.4149 - val_acc: 0.4105
Epoch 6/10000
97538/97538 [==============================] - 20s - loss: 0.3603 - acc: 0.8437 - val_loss: 1.2477 - val_acc: 0.4717
Epoch 7/10000
97538/97538 [==============================] - 20s - loss: 0.3366 - acc: 0.8603 - val_loss: 0.9204 - val_acc: 0.6512
Epoch 8/10000
97538/97538 [=

97454/97454 [==============================] - 20s - loss: 0.3334 - acc: 0.8610 - val_loss: 0.3285 - val_acc: 0.8585
Epoch 10/10000
97454/97454 [==============================] - 20s - loss: 0.3276 - acc: 0.8630 - val_loss: 0.3383 - val_acc: 0.8536
Epoch 11/10000
97454/97454 [==============================] - 20s - loss: 0.3254 - acc: 0.8633 - val_loss: 0.4407 - val_acc: 0.8166
Epoch 12/10000
97454/97454 [==============================] - 20s - loss: 0.3245 - acc: 0.8639 - val_loss: 0.3097 - val_acc: 0.8584
Epoch 13/10000
97454/97454 [==============================] - 20s - loss: 0.3202 - acc: 0.8668 - val_loss: 0.3270 - val_acc: 0.8527
Epoch 14/10000
97454/97454 [==============================] - 20s - loss: 0.3183 - acc: 0.8672 - val_loss: 0.3271 - val_acc: 0.8569
Epoch 15/10000
97454/97454 [==============================] - 20s - loss: 0.3169 - acc: 0.8668 - val_loss: 0.3870 - val_acc: 0.8207
Epoch 16/10000
97454/97454 [==============================] - 20s - loss: 0.3156 - acc: 0.8

97400/97400 [==============================] - 20s - loss: 0.2943 - acc: 0.8807 - val_loss: 0.3354 - val_acc: 0.8495
Epoch 41/10000
97400/97400 [==============================] - 20s - loss: 0.2917 - acc: 0.8813 - val_loss: 0.2797 - val_acc: 0.8650
Epoch 42/10000
97400/97400 [==============================] - 20s - loss: 0.2914 - acc: 0.8823 - val_loss: 0.3360 - val_acc: 0.8489
Epoch 43/10000
97400/97400 [==============================] - 20s - loss: 0.2907 - acc: 0.8827 - val_loss: 0.2798 - val_acc: 0.8720
Epoch 44/10000
97400/97400 [==============================] - 20s - loss: 0.2920 - acc: 0.8815 - val_loss: 0.3142 - val_acc: 0.8559
Epoch 45/10000
97400/97400 [==============================] - 20s - loss: 0.2898 - acc: 0.8831 - val_loss: 0.3040 - val_acc: 0.8633
Epoch 46/10000
97400/97400 [==============================] - 20s - loss: 0.2897 - acc: 0.8830 - val_loss: 0.2878 - val_acc: 0.8718
Epoch 47/10000
97280/97400 [============================>.] - ETA: 0sKF_7
Train on 97502 sa

97496/97496 [==============================] - 20s - loss: 0.3239 - acc: 0.8641 - val_loss: 0.2821 - val_acc: 0.8709
Epoch 15/10000
97496/97496 [==============================] - 20s - loss: 0.3226 - acc: 0.8663 - val_loss: 0.4276 - val_acc: 0.8381
Epoch 16/10000
97496/97496 [==============================] - 20s - loss: 0.3219 - acc: 0.8655 - val_loss: 0.4087 - val_acc: 0.8310
Epoch 17/10000
97496/97496 [==============================] - 20s - loss: 0.3220 - acc: 0.8668 - val_loss: 0.3525 - val_acc: 0.8428
Epoch 18/10000
97496/97496 [==============================] - 20s - loss: 0.3164 - acc: 0.8695 - val_loss: 0.3866 - val_acc: 0.8290
Epoch 19/10000
97496/97496 [==============================] - 20s - loss: 0.3169 - acc: 0.8688 - val_loss: 0.4098 - val_acc: 0.8258
Epoch 20/10000
97496/97496 [==============================] - 20s - loss: 0.3173 - acc: 0.8697 - val_loss: 0.3666 - val_acc: 0.8452
Epoch 21/10000
97496/97496 [==============================] - 21s - loss: 0.3142 - acc: 0.8

In [10]:
print(model_name)

metrics_kf

2 CNN 2 MAX SM


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.952206,0.794675,0.794675,0.557576,0.692168,0.567937,0.852295,00:11:32.252369
KF_2,0.949063,0.815070,0.815070,0.573122,0.744056,0.591021,0.860187,00:09:08.545531
KF_3,0.955923,0.770320,0.773642,0.556141,0.707625,0.561928,0.836096,00:17:15.154238
KF_4,0.952836,0.808530,0.811853,0.569640,0.720962,0.587120,0.864772,00:15:00.826103
KF_5,0.950674,0.779239,0.779239,0.562836,0.689092,0.578203,0.867798,00:15:49.524593
KF_6,0.953005,0.808120,0.808120,0.561229,0.726583,0.569550,0.838858,00:16:03.296284
KF_7,0.949540,0.798019,0.801342,0.554519,0.718724,0.555266,0.820705,00:08:12.071415
KF_8,0.951605,0.779345,0.779345,0.558789,0.694642,0.569977,0.853986,00:10:35.280690
KF_9,0.955237,0.788985,0.788985,0.562472,0.718343,0.573806,0.848836,00:14:53.482862
KF_10,0.947528,0.772844,0.776177,0.559698,0.714005,0.568706,0.843573,00:15:06.364188


# Metrics

In [11]:
models

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
2 CNN 1 MAX RO,0.950505,0.732790,0.733122,0.552522,0.706212,0.552968,0.822625,00:09:32.556510
2 CNN 2 MAX RO,0.885206,0.785091,0.786087,0.543794,0.729112,0.512764,0.740174,00:16:05.429269
2 CNN 1 MAX SM,0.976586,0.767251,0.768248,0.565410,0.699105,0.581455,0.867153,00:09:51.403921
2 CNN 2 MAX SM,0.951762,0.791515,0.792845,0.561602,0.712620,0.572351,0.848711,00:13:21.679827
